<a href="https://colab.research.google.com/github/axndyg/sagb_inrto_project/blob/main/Fouls_Cleaning_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
France = pd.read_csv("/content/events_France.csv")
France.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,tagsList,pos_orig_y,pos_orig_x,pos_dest_y,pos_dest_x
0,8,Simple pass,[{'id': 1801}],253784,"[{'y': 51, 'x': 50}, {'y': 46, 'x': 31}]",2500686,Pass,3799,1H,1.935181,85.0,176505119,[1801],51,50,46,31
1,8,High pass,[{'id': 1801}],29474,"[{'y': 46, 'x': 31}, {'y': 74, 'x': 68}]",2500686,Pass,3799,1H,3.599295,83.0,176505121,[1801],46,31,74,68
2,1,Air duel,"[{'id': 703}, {'id': 1801}]",253784,"[{'y': 74, 'x': 68}, {'y': 54, 'x': 72}]",2500686,Duel,3799,1H,6.827043,10.0,176505122,"[703, 1801]",74,68,54,72
3,1,Air duel,"[{'id': 701}, {'id': 1802}]",56441,"[{'y': 26, 'x': 32}, {'y': 46, 'x': 28}]",2500686,Duel,3772,1H,6.985577,10.0,176505017,"[701, 1802]",26,32,46,28
4,1,Ground attacking duel,"[{'id': 702}, {'id': 1801}]",366760,"[{'y': 54, 'x': 72}, {'y': 55, 'x': 73}]",2500686,Duel,3799,1H,9.511272,11.0,176505124,"[702, 1801]",54,72,55,73


In [ ]:
fouls_France = France[France["eventName"] == "Foul"]
fouls_France = fouls_France[['eventId', 'subEventName', 'matchId', 'teamId', 'matchPeriod', 'eventSec']]
fouls_France.head()

,eventId,subEventName,matchId,teamId,matchPeriod,eventSec
38,2,Foul,2500686,3799,1H,68.621397
70,2,Foul,2500686,3772,1H,173.279043
271,2,Foul,2500686,3799,1H,780.411047
397,2,Foul,2500686,3772,1H,1189.224805
424,2,Foul,2500686,3799,1H,1259.367422


In [ ]:
fouls_France.loc[fouls_France["matchPeriod"] == '2H', 'eventSec'] += 2400

fouls_France = fouls_France[['eventId', 'subEventName', 'matchId', 'teamId', 'eventSec']]
fouls_France['eventMin'] = round(fouls_France['eventSec'] / 60)
fouls_France.head()

,eventId,subEventName,matchId,teamId,eventSec,eventMin
38,2,Foul,2500686,3799,68.621397,1.0
70,2,Foul,2500686,3772,173.279043,3.0
271,2,Foul,2500686,3799,780.411047,13.0
397,2,Foul,2500686,3772,1189.224805,20.0
424,2,Foul,2500686,3799,1259.367422,21.0


In [ ]:
by_game = dict((fouls_France.groupby("matchId").apply(lambda x: list(dict(x["eventMin"]).values()))))


In [ ]:
fouls_cleaned = pd.DataFrame({'matchId': list(by_game.keys()), 'fouls': list(by_game.values())})


In [ ]:
fouls_per_game = fouls_cleaned['fouls'].apply(len).tolist()
fouls_by_game_late = fouls_cleaned['fouls'].apply(lambda x: len([i for i in x if i >= 45])).tolist()
fouls_by_game_first = fouls_cleaned['fouls'].apply(lambda x: len([i for i in x if i < 45])).tolist()


In [ ]:
fouls_cleaned["total fouls"] = fouls_per_game
fouls_cleaned["fouls in second half"] = fouls_by_game_late
fouls_cleaned["fouls in first half"] = fouls_by_game_first

fouls_cleaned["first half foul density"] =  fouls_cleaned["fouls in first half"] / fouls_cleaned["total fouls"]
fouls_cleaned["second half foul density"] =  fouls_cleaned["fouls in second half"] / fouls_cleaned["total fouls"]

In [ ]:
fouls_cleaned

,matchId,fouls,total fouls,fouls in second half,fouls in first half,first half foul density,second half foul density
0,2500686,"[1.0, 3.0, 13.0, 20.0, 21.0, 23.0, 31.0, 31.0,...",32,19,13,0.406250,0.593750
1,2500687,"[9.0, 13.0, 17.0, 23.0, 25.0, 27.0, 32.0, 35.0...",29,19,10,0.344828,0.655172
2,2500688,"[10.0, 10.0, 15.0, 24.0, 27.0, 28.0, 30.0, 36....",26,13,13,0.500000,0.500000
3,2500689,"[1.0, 6.0, 7.0, 12.0, 14.0, 15.0, 35.0, 42.0, ...",18,9,9,0.500000,0.500000
4,2500690,"[6.0, 7.0, 17.0, 19.0, 19.0, 27.0, 33.0, 33.0,...",21,8,13,0.619048,0.380952
...,...,...,...,...,...,...,...
375,2501061,"[6.0, 8.0, 16.0, 27.0, 35.0, 39.0, 44.0, 44.0,...",18,10,8,0.444444,0.555556
376,2501062,"[10.0, 19.0, 21.0, 23.0, 27.0, 28.0, 29.0, 31....",22,9,13,0.590909,0.409091
377,2501063,"[8.0, 14.0, 15.0, 20.0, 35.0, 38.0, 46.0, 53.0...",11,5,6,0.545455,0.454545
378,2501064,"[2.0, 4.0, 7.0, 16.0, 18.0, 23.0, 30.0, 33.0, ...",23,12,11,0.478261,0.521739


In [ ]:
fouls_France.to_csv("fouls_France.csv")

In [ ]:
fouls_cleaned.to_csv("fouls_cleaned.csv")